In [15]:
# Initial imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import date
from dateutil.relativedelta import relativedelta


In [4]:
# Load .env enviroment variables
load_dotenv()

True

In [93]:
# Set Universal variables:
selected_company="TSLA"
bench_mark="SPY"

today=date.today().strftime('%Y-%m-%d')


'2017-04-13'

In [ ]:
# function to get the date of "n years ago from today"
def n_year_ago(n):
    
    n_years_ago=(date.today() - relativedelta(years=n)).strftime('%Y-%m-%d')
    return n_years_ago


In [46]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [114]:
# Format current date as ISO format
start_date = pd.Timestamp(n_year_ago(6), tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

# Set the tickers list to fetch info from Alpaca API

tickers = [selected_company, bench_mark]


# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices in tickers list
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df


# Create a function to calculate returns
* daily return
* 1-year return 
* 5-year return 
* accumulative return from 1 year prior
* accumulative return from 5 year prior

In [115]:


ticker='TSLA'
df_ticker=df_ticker[['close','symbol']].loc[df_ticker['symbol']==ticker].drop(columns='symbol')

# create daily return 
df_ticker['daily_return']=df_ticker['close'].pct_change()

# create 1-year return 
df_ticker['1year_return']=df_ticker['close'].pct_change(periods=252*1)

# create 5-year return 
df_ticker['5year_return']=df_ticker['close'].pct_change(periods=252*5)
df_ticker['5year_return_annualzd']=(1+df_ticker[['5year_return']])**(1/5)-1

In [116]:

# create 1-year cumulative return
df_ticker_1=df_ticker.loc[n_year_ago(1):]
df_ticker_1['1yr cumulative return'] = (1 + df_ticker_1['daily_return']).cumprod() - 1  
df_ticker_1

/Users/feiyiyang/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,close,daily_return,1year_return,5year_return,5year_return_annualzd,1yr cumulative return
timestamp,,,,,,
2021-04-14 04:00:00+00:00,732.730,-0.038854,0.032174,NaN,NaN,-0.038854
2021-04-15 04:00:00+00:00,738.700,0.008148,0.012154,NaN,NaN,-0.031022
2021-04-16 04:00:00+00:00,739.395,0.000941,-0.007803,1.935738,0.240348,-0.030111
2021-04-19 04:00:00+00:00,714.980,-0.033020,-0.051612,1.809241,0.229470,-0.062137
2021-04-20 04:00:00+00:00,718.790,0.005329,-0.036939,1.831219,0.231387,-0.057139
...,...,...,...,...,...,...
2022-04-07 04:00:00+00:00,1057.260,0.010997,0.561638,2.539538,0.287625,0.386843
2022-04-08 04:00:00+00:00,1025.490,-0.030049,0.462687,2.389601,0.276526,0.345170
2022-04-11 04:00:00+00:00,975.930,-0.048328,0.280160,2.124076,0.255869,0.280160


In [ ]:


# create 5-year cumulative return


# drop NA rows
df_ticker_1.dropna(inplace=True)
df_ticker_1.tail()
